In [ ]:
import pandas as pd
import plotly.express as px
import os

# === 1. Set base path to datasets ===
datasets_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets"

# === 2. Friendly zone names ===
zone_map = {
    'LZ_HOUSTON': 'Houston Metro',
    'LZ_NORTH': 'Dallas–Fort Worth',
    'LZ_SOUTH': 'South Texas',
    'LZ_WEST': 'West Texas',
    'LZ_AEN': 'Corpus Christi (AEP North)',
    'LZ_CPS': 'San Antonio (CPS Energy)',
    'LZ_LCRA': 'Central Texas (LCRA)',
    'LZ_RAYBN': 'Northeast TX (Rayburn)',
    'LZ_SOUTHERN': 'South Central TX',
    'LZ_FEPC': 'Panhandle (FEPC)',
    'LZ_ONCOR': 'Dallas Suburban (Oncor)'
}


# === 3. Plot ERCOT Total Actual Load ===
al_df = pd.read_csv(os.path.join(datasets_path, "AL_combined.csv"))
al_df['Date'] = pd.to_datetime(al_df['Date'], errors='coerce')

fig_al = px.line(
    al_df,
    x='Date',
    y='Load',
    title='ERCOT Total Actual Load (MW)',
    labels={'Load': 'Load (MW)', 'Date': 'Time'}
)
fig_al.update_layout(legend_title_text=None)
fig_al.show()

# === 4. Plot RTM Prices ===
rtm_df = pd.read_csv(os.path.join(datasets_path, "RTM_combined.csv"))
rtm_df['Date'] = pd.to_datetime(rtm_df['Date'], errors='coerce')
rtm_df['Zone'] = rtm_df['Zone'].map(zone_map).fillna(rtm_df['Zone'])

fig_rtm = px.line(
    rtm_df,
    x='Date',
    y='Price',
    color='Zone',
    title='ERCOT Real-Time Market Prices by Zone',
    labels={'Price': 'Price ($/MWh)', 'Date': 'Time', 'Zone': 'Zone'}
)
fig_rtm.update_layout(legend_title_text='Zone')
fig_rtm.show()


# === 5. Plot DAM Prices ===
dam_df = pd.read_csv(os.path.join(datasets_path, "DAM_combined.csv"))
dam_df['Date'] = pd.to_datetime(dam_df['Date'], errors='coerce')
dam_df['zone'] = dam_df['zone'].map(zone_map).fillna(dam_df['zone'])

fig_dam = px.line(
    dam_df,
    x='Date',
    y='price',
    color='zone',
    title='ERCOT Day-Ahead Market Prices by Zone',
    labels={'price': 'Price ($/MWh)', 'Date': 'Time', 'zone': 'Zone'}
)
fig_dam.update_layout(legend_title_text='Zone')
fig_dam.show()


# === 6. Plot Ancillary Services MCP ===
as_df = pd.read_csv(os.path.join(datasets_path, "AS_combined.csv"))
as_df['Date'] = pd.to_datetime(as_df['Date'], errors='coerce')

fig_as = px.line(
    as_df,
    x='Date',
    y='mcpc',
    color='ancillarytype',
    title='ERCOT Ancillary Services MCP by Type',
    labels={'mcpc': 'MCP ($/MWh)', 'Date': 'Time', 'ancillarytype': 'Service Type'}
)
fig_as.update_layout(legend_title_text='Ancillary Service')
fig_as.show()

# === 7. Plot BESS Net Output ===
bess_df = pd.read_csv(os.path.join(datasets_path, "BESS_combined.csv"))
bess_df['time_local'] = pd.to_datetime(bess_df['time_local'], errors='coerce')

fig_net = px.line(
    bess_df,
    x='time_local',
    y='net_output',
    title='Net BESS Output Over Time',
    labels={'time_local': 'Time', 'net_output': 'Net Output (MW)'}
)
fig_net.show()

# === 8. Plot BESS Charging vs Discharging (Stacked Area) ===
fig_stack = px.area(
    bess_df,
    x='time_local',
    y=['total_charging', 'total_discharging'],
    title='BESS Charging and Discharging (MW)',
    labels={'value': 'Power (MW)', 'time_local': 'Time', 'variable': 'Flow Type'}
)
fig_stack.show()


In [ ]:
import pandas as pd
import plotly.express as px
import os

# === 1. Set base path to datasets ===
datasets_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets"

# === 2. Define 4 main ERCOT congestion zones map ===
zone_map = {
    'LZ_HOUSTON': 'Houston Metro',
    'LZ_NORTH': 'Dallas–Fort Worth',
    'LZ_SOUTH': 'South Texas',
    'LZ_WEST': 'West Texas',
    'LZ_AEN': 'South Texas',
    'LZ_CPS': 'South Texas',
    'LZ_LCRA': 'South Texas',
    'LZ_RAYBN': 'Dallas–Fort Worth',
    'LZ_SOUTHERN': 'South Texas',
    'LZ_FEPC': 'West Texas',
    'LZ_ONCOR': 'Dallas–Fort Worth'
}

# === Helper function: filter from 2016 and rolling smooth ===
def filter_and_smooth(df, date_col, y_col, group_cols=None, window=7):
    df = df[df[date_col] >= '2016-01-01'].copy()
    if group_cols:
        df = df.groupby([pd.Grouper(key=date_col, freq='D')] + group_cols)[y_col].mean().reset_index()
        df[y_col] = df.groupby(group_cols)[y_col].transform(lambda x: x.rolling(window, min_periods=1).mean())
    else:
        df = df.groupby(pd.Grouper(key=date_col, freq='D'))[y_col].mean().reset_index()
        df[y_col] = df[y_col].rolling(window, min_periods=1).mean()
    return df

# === 3. ERCOT Total Actual Load (MW) ===
al_df = pd.read_csv(os.path.join(datasets_path, "AL_combined.csv"))
al_df['Date'] = pd.to_datetime(al_df['Date'], errors='coerce')
al_df = filter_and_smooth(al_df, 'Date', 'Load')

fig_al = px.line(
    al_df,
    x='Date',
    y='Load',
    title='ERCOT Total Actual Load (MW)',
    labels={'Load': 'Load (MW)', 'Date': 'Time'}
)
fig_al.update_layout(legend_title_text=None)
fig_al.show()

# === 4. RTM Prices by Congestion Zone ===
rtm_df = pd.read_csv(os.path.join(datasets_path, "RTM_combined.csv"))
rtm_df['Date'] = pd.to_datetime(rtm_df['Date'], errors='coerce')
rtm_df['Zone'] = rtm_df['Zone'].map(zone_map).fillna(rtm_df['Zone'])
rtm_df = filter_and_smooth(rtm_df, 'Date', 'Price', ['Zone'])

fig_rtm = px.line(
    rtm_df,
    x='Date',
    y='Price',
    color='Zone',
    title='ERCOT Real-Time Market Prices by Congestion Zone',
    labels={'Price': 'Price ($/MWh)', 'Date': 'Time', 'Zone': 'Zone'}
)
fig_rtm.update_layout(legend_title_text='Zone')
fig_rtm.show()

# === 5. DAM Prices by Congestion Zone ===
dam_df = pd.read_csv(os.path.join(datasets_path, "DAM_combined.csv"))
dam_df['Date'] = pd.to_datetime(dam_df['Date'], errors='coerce')
dam_df['zone'] = dam_df['zone'].map(zone_map).fillna(dam_df['zone'])
dam_df = filter_and_smooth(dam_df, 'Date', 'price', ['zone'])

fig_dam = px.line(
    dam_df,
    x='Date',
    y='price',
    color='zone',
    title='ERCOT Day-Ahead Market Prices by Congestion Zone',
    labels={'price': 'Price ($/MWh)', 'Date': 'Time', 'zone': 'Zone'}
)
fig_dam.update_layout(legend_title_text='Zone')
fig_dam.show()

# === 6. Ancillary Services MCP (7-day rolling) ===
as_df = pd.read_csv(os.path.join(datasets_path, "AS_combined.csv"))
as_df['Date'] = pd.to_datetime(as_df['Date'], errors='coerce')
as_df = filter_and_smooth(as_df, 'Date', 'mcpc', ['ancillarytype'])

fig_as = px.line(
    as_df,
    x='Date',
    y='mcpc',
    color='ancillarytype',
    title='ERCOT Ancillary Services MCP (7-day avg)',
    labels={'mcpc': 'MCP ($/MWh)', 'Date': 'Time', 'ancillarytype': 'Service Type'}
)
fig_as.update_layout(legend_title_text='Ancillary Service')
fig_as.show()

# === 7. Net BESS Output (7-day smoothed) ===
bess_df = pd.read_csv(os.path.join(datasets_path, "BESS_combined.csv"))
bess_df['time_local'] = pd.to_datetime(bess_df['time_local'], errors='coerce')
bess_df = filter_and_smooth(bess_df, 'time_local', 'net_output')

fig_net = px.line(
    bess_df,
    x='time_local',
    y='net_output',
    title='Net BESS Output Over Time (7-day avg)',
    labels={'time_local': 'Time', 'net_output': 'Net Output (MW)'}
)
fig_net.show()

# === 8. BESS Charging and Discharging (7-day smoothed stacked area) ===
bess_df = pd.read_csv(os.path.join(datasets_path, "BESS_combined.csv"))
bess_df['time_local'] = pd.to_datetime(bess_df['time_local'], errors='coerce')
bess_df = bess_df[bess_df['time_local'] >= '2016-01-01']
bess_df['total_charging'] = bess_df['total_charging'].rolling(7, min_periods=1).mean()
bess_df['total_discharging'] = bess_df['total_discharging'].rolling(7, min_periods=1).mean()

fig_stack = px.area(
    bess_df,
    x='time_local',
    y=['total_charging', 'total_discharging'],
    title='BESS Charging and Discharging (7-day avg)',
    labels={'value': 'Power (MW)', 'time_local': 'Time', 'variable': 'Flow Type'}
)
fig_stack.show()


In [ ]:
import pandas as pd
import plotly.express as px
import os

# === Path to dataset ===
datasets_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets"
al_path = os.path.join(datasets_path, "AL_combined.csv")

# === Load and preprocess ===
al_df = pd.read_csv(al_path)
al_df['Date'] = pd.to_datetime(al_df['Date'], errors='coerce')
al_df = al_df[al_df['Date'] >= '2016-01-01'].copy()

# === Daily aggregation and smoothing ===
al_df = al_df.groupby('Date')['Load'].mean().reset_index()
al_df['Load'] = al_df['Load'].rolling(window=7, min_periods=1).mean()

# === Plot ===
fig_al = px.line(
    al_df,
    x='Date',
    y='Load',
    title='ERCOT Total Actual Load (7-Day Rolling Avg)',
    labels={'Load': 'Load (MW)', 'Date': 'Date'}
)
fig_al.update_layout(
    legend_title_text=None,
    width=1100,
    height=500
)
fig_al.show()


In [ ]:
import pandas as pd
import plotly.express as px
import os

# === File path ===
datasets_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets"
rtm_path = os.path.join(datasets_path, "RTM_combined.csv")

# === Official ERCOT Load Zone Names ===
zone_map = {
    'LZ_NORTH': 'North',
    'LZ_SOUTH': 'South',
    'LZ_WEST': 'West',
    'LZ_HOUSTON': 'Houston'
}

# === Load and preprocess ===
df = pd.read_csv(rtm_path)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df[df['Date'] >= '2016-01-01']
df = df[df['Zone'].isin(zone_map)].copy()
df['Zone'] = df['Zone'].map(zone_map)

# === Apply 7-day rolling average ===
df = df.sort_values(['Zone', 'Date'])
df['Smoothed Price'] = df.groupby('Zone')['Price'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# === Faceted Line Plot ===
fig = px.line(
    df,
    x='Date',
    y='Smoothed Price',
    facet_col='Zone',
    facet_col_wrap=2,
    title='ERCOT RTM Prices by Load Zone (7-Day Rolling Average)',
    labels={'Smoothed Price': 'Price ($/MWh)', 'Date': 'Date'}
)

fig.update_layout(
    height=600,
    width=1200,
    showlegend=False,
    margin=dict(t=60, b=40),
)

# Simplify facet titles
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# === Load DAM data ===
dam_df = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\DAM_combined.csv")
dam_df['Date'] = pd.to_datetime(dam_df['Date'], errors='coerce')

# === Filter to post-2016 and key zones ===
zones = ['LZ_HOUSTON', 'LZ_NORTH', 'LZ_SOUTH', 'LZ_WEST']
dam_df = dam_df[(dam_df['Date'] >= '2016-01-01') & (dam_df['zone'].isin(zones))]

# === Friendly zone names ===
zone_map = {
    'LZ_HOUSTON': 'Houston',
    'LZ_NORTH': 'North',
    'LZ_SOUTH': 'South',
    'LZ_WEST': 'West'
}
dam_df['Zone'] = dam_df['zone'].map(zone_map)

# === Rolling average for smoothing ===
dam_df = dam_df.sort_values(['Zone', 'Date'])
dam_df['Price_avg'] = dam_df.groupby('Zone')['price'].transform(lambda x: x.rolling(7, min_periods=1).mean())

# === Faceted plot by Zone ===
fig = px.line(
    dam_df,
    x='Date',
    y='Price_avg',
    facet_col='Zone',
    facet_col_wrap=2,
    title='ERCOT DAM Prices by Load Zone (7-Day Rolling Average)',
    labels={'Price_avg': 'Price ($/MWh)', 'Date': 'Date'},
    color_discrete_sequence=['#636EFA']
)

fig.update_layout(height=600, width=1000)
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# === Load AS data ===
as_df = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\AS_combined.csv")
as_df['Date'] = pd.to_datetime(as_df['Date'], errors='coerce')

# === Define custom order with empty placeholder ===
ordered_types = [
    'ECRS',
    'Non Spin',
    'Regulation Down',
    'Regulation Up',
    '␣',  # Placeholder for empty plot
    'Responsive Reserve Service'
]

# Add placeholder row for blank subplot
placeholder = pd.DataFrame({
    'Date': pd.to_datetime(['2016-01-01']),
    'ancillarytype': ['␣'],
    'mcpc': [None],
    'MCP_avg': [None]
})
as_df = pd.concat([as_df, placeholder], ignore_index=True)

# Filter and apply category order
as_df = as_df[as_df['ancillarytype'].isin(ordered_types)]
as_df['ancillarytype'] = pd.Categorical(as_df['ancillarytype'], categories=ordered_types, ordered=True)

# Rolling avg
as_df = as_df.sort_values(['ancillarytype', 'Date'])
as_df['MCP_avg'] = as_df.groupby('ancillarytype')['mcpc'].transform(lambda x: x.rolling(7, min_periods=1).mean())

# === Plot ===
fig = px.line(
    as_df,
    x='Date',
    y='MCP_avg',
    facet_col='ancillarytype',
    facet_col_wrap=2,
    title='ERCOT Ancillary Services MCP by Type (7-Day Rolling Avg)',
    labels={'MCP_avg': 'MCP ($/MWh)', 'Date': 'Date'},
    color_discrete_sequence=['#636EFA']
)

# Remove "=␣" and clean facet titles
fig.for_each_annotation(lambda a: a.update(text='' if a.text.endswith('␣') else a.text.split("=")[-1]))

# Format all x-axes to show years
fig.for_each_xaxis(lambda axis: axis.update(tickformat='%Y'))

# Style layout
fig.update_layout(height=1000, width=1000)
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load actual load data
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\AL_combined.csv"
al_df = pd.read_csv(file_path, parse_dates=['Date'])
al_df = al_df.dropna(subset=['Date', 'Load'])

# Filter data from 2016 onwards
al_df = al_df[al_df['Date'] >= '2016-01-01']

# Extract hour and year
al_df['Hour'] = al_df['Date'].dt.hour
al_df['Year'] = al_df['Date'].dt.year

# Group by year and hour to compute mean load
hourly_avg = al_df.groupby(['Year', 'Hour'])['Load'].mean().reset_index()
pivot_df = hourly_avg.pivot(index='Hour', columns='Year', values='Load')

# Define highlight colors for 2023, 2024, 2025
highlight_years = {
    2023: '#4e79a7',  # medium blue
    2024: '#2f5597',  # navy blue
    2025: '#103f75'   # darker navy
}

# Create plotly figure
fig = go.Figure()

for year in pivot_df.columns:
    color = highlight_years.get(year, 'lightgrey')
    width = 2.5 if year in highlight_years else 1
    fig.add_trace(go.Scatter(
        x=pivot_df.index,
        y=pivot_df[year],
        mode='lines',
        name=str(year),
        line=dict(color=color, width=width)
    ))

fig.update_layout(
    title="ERCOT Average Daily Load Profile by Year (24h)",
    xaxis_title="Hour of Day",
    yaxis_title="Average Load (MW)",
    xaxis=dict(tickmode='linear', dtick=1),
    legend_title="Year",
    height=500,
    width=900
)

fig.show()


In [2]:
import pandas as pd

# === Load file ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_combined.csv"
df = pd.read_csv(file_path)

# === Ensure datetime parsing ===
df['time_utc'] = pd.to_datetime(df['time_utc'], errors='coerce')

# === Drop rows with invalid or missing datetimes ===
df = df.dropna(subset=['time_utc'])

# === Create separate Date and Hour columns ===
df['Date'] = df['time_utc'].dt.date
df['Hour'] = df['time_utc'].dt.hour
df['Minute'] = df['time_utc'].dt.minute
df['Timestamp'] = df['time_utc']  # if needed for full datetime

# === View the cleaned data ===
print(df[['Timestamp', 'Date', 'Hour', 'Minute', 'total_charging', 'total_discharging', 'net_output']].head())


                  Timestamp        Date  Hour  Minute  total_charging  \
0 2024-01-01 06:00:00+00:00  2024-01-01     6       0         -34.447   
1 2024-01-01 06:05:00+00:00  2024-01-01     6       5        -142.238   
2 2024-01-01 06:10:00+00:00  2024-01-01     6      10        -242.668   
3 2024-01-01 06:15:00+00:00  2024-01-01     6      15        -271.211   
4 2024-01-01 06:20:00+00:00  2024-01-01     6      20        -294.958   

   total_discharging  net_output  
0             13.854     -20.593  
1             41.540    -100.698  
2             15.229    -227.438  
3             12.088    -259.122  
4              9.225    -285.733  


In [3]:
import pandas as pd
import plotly.graph_objects as go

# === Load the data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_combined.csv"
df = pd.read_csv(file_path)

# === Parse datetime and preprocess ===
df['time_utc'] = pd.to_datetime(df['time_utc'], errors='coerce')
df = df.dropna(subset=['time_utc'])

df['Month'] = df['time_utc'].dt.to_period('M').dt.to_timestamp()

# === Monthly aggregation ===
monthly_df = df.groupby('Month')[['total_charging', 'total_discharging']].sum().reset_index()
monthly_df['Charging_MWh'] = monthly_df['total_charging'].abs() / 1000
monthly_df['Discharging_MWh'] = monthly_df['total_discharging'] / 1000

# === Plot ===
fig = go.Figure()

fig.add_trace(go.Bar(
    x=monthly_df['Month'],
    y=monthly_df['Discharging_MWh'],
    name='Discharging (MWh)',
    marker_color='mediumpurple'
))

fig.add_trace(go.Bar(
    x=monthly_df['Month'],
    y=monthly_df['Charging_MWh'],
    name='Charging (MWh)',
    marker_color='thistle'
))

fig.update_layout(
    barmode='group',  # SIDE BY SIDE
    title='Monthly BESS Energy Throughput (Charging vs Discharging)',
    xaxis_title='Month',
    yaxis_title='Energy (MWh)',
    legend_title='Flow Type',
    height=600,
    width=1000
)

fig.show()


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_1824\223785692.py:12: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Month'] = df['time_utc'].dt.to_period('M').dt.to_timestamp()


In [9]:
# Round timestamps to the hour
df['hour'] = df['time_utc'].dt.floor('H')

# Count unique hours
n_hours = df['hour'].nunique()


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_1824\362005243.py:2: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [10]:
# Sum MWh
total_disch_mwh = df['total_discharging'].sum()
total_charg_mwh = df['total_charging'].abs().sum()

# Compute average MW over the number of hours
avg_disch_mw = total_disch_mwh / n_hours
avg_charg_mw = total_charg_mwh / n_hours

# Convert to GW
avg_disch_gw = avg_disch_mw / 1000
avg_charg_gw = avg_charg_mw / 1000

print(f"🔋 Avg BESS discharging capacity: {avg_disch_gw:.2f} GW")
print(f"🔌 Avg BESS charging capacity: {avg_charg_gw:.2f} GW")


🔋 Avg BESS discharging capacity: 3.80 GW
🔌 Avg BESS charging capacity: 4.67 GW


In [ ]:
import pandas as pd
import plotly.express as px

# === Load data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_combined.csv"
df = pd.read_csv(file_path)

# === Convert to datetime ===
df['time_utc'] = pd.to_datetime(df['time_utc'], errors='coerce')
df = df.dropna(subset=['time_utc'])

# === Extract hour and year ===
df['Hour'] = df['time_utc'].dt.hour
df['Year'] = df['time_utc'].dt.year

# === Group by hour and year ===
hourly = df.groupby(['Year', 'Hour'])['total_discharging'].mean().reset_index()

# === Define colors ===
highlight_years = {2023: '#4e79a7', 2024: '#2f5597', 2025: '#103f75'}

# === Plot with Plotly ===
fig = px.line(
    hourly,
    x='Hour',
    y='total_discharging',
    color='Year',
    labels={'total_discharging': 'Avg Discharging (MWh)', 'Hour': 'Hour of Day'},
    title='Average BESS Discharging by Hour (Daily Profile)'
)

# === Update color manually for 2023–2025, others in grey ===
for trace in fig.data:
    year = int(trace.name)
    if year in highlight_years:
        trace.line.color = highlight_years[year]
        trace.line.width = 2.5
    else:
        trace.line.color = 'lightgrey'
        trace.line.width = 1.0

fig.update_layout(height=500, width=900)
fig.show()


In [ ]:
import pandas as pd
import plotly.express as px
import os

# === Set paths ===
datasets_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets"
zone_map = {
    'LZ_HOUSTON': 'Houston',
    'LZ_NORTH': 'North',
    'LZ_SOUTH': 'South',
    'LZ_WEST': 'West',
}

# === Load RTM data for 2024 and prep ===
rtm_df = pd.read_csv(os.path.join(datasets_path, "RTM_combined.csv"))
rtm_df['Date'] = pd.to_datetime(rtm_df['Date'], errors='coerce', utc=True).dt.tz_convert(None)
rtm_df = rtm_df[(rtm_df['Date'].dt.year == 2024) & (rtm_df['Zone'].isin(zone_map.keys()))]
rtm_df['Zone'] = rtm_df['Zone'].map(zone_map)
rtm_df['Hour'] = rtm_df['Date'].dt.floor('H')

# === Compute daily 2-hour spreads ===
spread_df = (
    rtm_df.assign(DateOnly=rtm_df['Hour'].dt.date)
    .groupby(['Zone', 'DateOnly'])['Price']
    .agg(lambda x: x.nlargest(2).mean() - x.nsmallest(2).mean())
    .reset_index(name='Spread')
)
spread_df['Month'] = pd.to_datetime(spread_df['DateOnly']).dt.to_period('M').dt.to_timestamp()
monthly_spread = spread_df.groupby(['Zone', 'Month'])['Spread'].mean().reset_index()
zone_avg = monthly_spread.groupby('Zone')['Spread'].mean().reset_index(name='AvgSpread')

# === Plot monthly spreads with dashed avg lines ===
fig = px.bar(monthly_spread, x='Month', y='Spread', color='Zone', barmode='group',
             title='Monthly Average 2-Hour Price Spreads by Zone (2024)',
             labels={'Spread': '2-Hour Spread ($/MWh)', 'Month': 'Month'})
for i, row in zone_avg.iterrows():
    fig.add_hline(y=row.AvgSpread, line_dash="dash",
                  annotation_text=f"{row.Zone} Avg: ${row.AvgSpread:.0f}/MWh",
                  line_color=px.colors.qualitative.Plotly[i])
fig.update_layout(xaxis_tickformat='%b %Y', legend_title_text='Zone')
fig.show()



In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Load the file
file_path = "C:/Users/amina.talipova/Desktop/ercot/datasets/FM_combined.csv"
df = pd.read_csv(file_path, parse_dates=["Datetime"])

# Clean and unify fuel labels
df['Fuel'] = df['Fuel'].str.strip().str.title()
df['Fuel'] = df['Fuel'].replace({
    'Gas-Cc': 'Gas',
    'Gas-CC': 'Gas',
    'Wsl': 'Wind'
})

# Keep only valid fuels
valid_fuels = ['Coal', 'Gas', 'Nuclear', 'Wind', 'Solar', 'Hydro', 'Biomass', 'Other']
df = df[df['Fuel'].isin(valid_fuels)]

# Monthly grouping and conversion to GW
df['Month'] = df['Datetime'].dt.to_period('M').dt.to_timestamp()
df['GW'] = df['MW'] / 1000  # convert to GW

monthly = df.groupby(['Month', 'Fuel'])['GW'].sum().reset_index()

# Pivot for area plot and renewables line
pivot = monthly.pivot(index='Month', columns='Fuel', values='GW').fillna(0)
pivot['Total'] = pivot.sum(axis=1)
renewables = ['Wind', 'Solar', 'Hydro', 'Biomass']
pivot['Renewables'] = pivot[renewables].sum(axis=1)
pivot['Renewables_%'] = (pivot['Renewables'] / pivot['Total']) * 100

# === Area plot ===
fig = go.Figure()

color_map = {
    'Coal': '#636EFA',
    'Gas': '#EF553B',
    'Nuclear': '#00CC96',
    'Wind': '#AB63FA',
    'Solar': '#FFA15A',
    'Hydro': '#19D3F3',
    'Biomass': '#B6E880',
    'Other': '#FF6692'
}

# Add stacked area for each fuel
for fuel in valid_fuels:
    if fuel in pivot.columns:
        fig.add_trace(go.Scatter(
            x=pivot.index, y=pivot[fuel],
            name=fuel,
            mode='lines',
            stackgroup='one',
            line=dict(width=0.5),
            fillcolor=color_map.get(fuel, None)
        ))

# Add renewables share line (right y-axis)
fig.add_trace(go.Scatter(
    x=pivot.index, y=pivot['Renewables_%'],
    name='Renewables Share (%)',
    mode='lines',
    line=dict(color='black', dash='dash'),
    yaxis='y2'
))

fig.update_layout(
    title='ERCOT Monthly Generation by Fuel (GW) with Renewables Share (%)',
    xaxis=dict(title='Month'),
    yaxis=dict(
        title='Total Generation (GW)',
        tickformat='.0f',
        rangemode='tozero'
    ),
    yaxis2=dict(
        title='Renewables Share (%)',
        overlaying='y',
        side='right',
        showgrid=False,
        range=[0, 100]
    ),
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=-0.35,
        xanchor='center',
        x=0.5
    ),
    width=1200,
    height=650
)

fig.show()

# === Yearly Summary Text ===
df['Year'] = df['Month'].dt.year
annual = df[df['Fuel'].isin(renewables)].groupby(['Year', 'Fuel'])['GW'].sum().unstack().fillna(0)
annual['Total Renewables'] = annual.sum(axis=1)
annual_share = df.groupby('Year')['GW'].sum()
annual['Renewables Share (%)'] = (annual['Total Renewables'] / annual_share) * 100

# Print below the chart
print("🟢 Yearly Renewable Generation Summary (in GW):\n")
display_df = annual.round(1)
print(display_df.to_string())


In [ ]:
import pandas as pd
import plotly.express as px
import os

# === Define paths and zone map ===
datasets_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets"
zone_map = {
    'LZ_WEST': 'West',
    'LZ_SOUTH': 'South',
    'LZ_HOUSTON': 'Houston',
    'LZ_NORTH': 'North'
}

# === Load RTM (hourly) and BESS (5-min) data ===
rtm_df = pd.read_csv(os.path.join(datasets_path, "RTM_combined.csv"))
bess_df = pd.read_csv(os.path.join(datasets_path, "BESS_combined.csv"))

# === Prepare RTM data ===
rtm_df['Date'] = pd.to_datetime(rtm_df['Date'], errors='coerce', utc=True).dt.tz_convert(None)
rtm_df['Zone'] = rtm_df['Zone'].map(zone_map)
rtm_df = rtm_df[rtm_df['Date'].dt.year == 2024]
rtm_df['Hour'] = rtm_df['Date'].dt.floor('H')

# === Prepare BESS data ===
bess_df['time_local'] = pd.to_datetime(bess_df['time_local'], errors='coerce', utc=True).dt.tz_convert(None)
bess_df = bess_df[bess_df['time_local'].dt.year == 2024]
bess_df['Hour'] = bess_df['time_local'].dt.floor('H')
bess_df['Energy_MWh'] = bess_df['net_output'] * (5 / 60)  # 5-min intervals to MWh

# === Revenue calculation logic ===
results = []

for zone in ['West', 'South', 'Houston', 'North']:
    zone_prices = rtm_df[rtm_df['Zone'] == zone][['Hour', 'Price']].copy()
    zone_prices['Price'] = pd.to_numeric(zone_prices['Price'], errors='coerce')
    
    merged = bess_df.merge(zone_prices, on='Hour', how='inner')
    merged = merged.dropna(subset=['Energy_MWh', 'Price'])
    merged['Revenue'] = merged['Energy_MWh'] * merged['Price']
    merged['Month'] = merged['Hour'].dt.to_period('M').dt.to_timestamp()
    
    monthly_rev = merged.groupby('Month')['Revenue'].sum().reset_index()
    monthly_rev['Zone'] = zone
    results.append(monthly_rev)

rev_df = pd.concat(results, ignore_index=True)

# === Annualized revenue per zone Jan–May ===
jan_may_df = rev_df[rev_df['Month'].dt.month <= 5]
annualized = (
    jan_may_df.groupby('Zone')['Revenue']
    .sum()
    .reset_index(name='AnnualizedRevenue')
)
annualized['AnnualizedRevenue'] *= (12 / 5)  # scale to 12 months
annualized['Label'] = annualized['AnnualizedRevenue'].apply(lambda x: f"${x/1000:.0f}k/MW/year")

# === Bar chart ===
fig = px.bar(
    annualized,
    x='Zone', y='AnnualizedRevenue',
    color='Zone',
    text='Label',
    title='Estimated Annualized Revenue per MW by Zone (Jan–May 2024)',
    labels={'AnnualizedRevenue': '$/MW/year'}
)
fig.update_traces(textposition='outside')
fig.update_layout(showlegend=False, yaxis_title="$/MW/year", xaxis_title=None)
fig.show()


# MASTER DATASET 

In [16]:
import pandas as pd

# === Load Data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\RTM_combined.csv"
df = pd.read_csv(file_path)

# === Convert 'Date' column to datetime ===
df['datetime'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %H:%M', errors='coerce')

# === Drop rows with invalid dates ===
df = df.dropna(subset=['datetime'])

# === Convert Price to numeric (just in case) ===
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

# === Average duplicates if same datetime-zone appears more than once ===
grouped = df.groupby(['datetime', 'Zone'], as_index=False)['Price'].mean()

# === Pivot so each zone becomes a column ===
reshaped = grouped.pivot(index='datetime', columns='Zone', values='Price').reset_index()

# === Save to file (optional) ===
output_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\RTM_hourly_cleaned.csv"
reshaped.to_csv(output_path, index=False)

print("✅ RTM reshaped and saved:", output_path)


✅ RTM reshaped and saved: C:\Users\amina.talipova\Desktop\ercot\datasets\RTM_hourly_cleaned.csv


In [17]:
import pandas as pd

# === Load DAM Data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\DAM_combined.csv"
df = pd.read_csv(file_path)

# === Convert Date column to datetime ===
df['datetime'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %H:%M', errors='coerce')

# === Drop rows with bad datetime ===
df = df.dropna(subset=['datetime'])

# === Convert price to numeric (if not already) ===
df['price'] = pd.to_numeric(df['price'], errors='coerce')

# === Group by datetime and zone to handle duplicates ===
grouped = df.groupby(['datetime', 'zone'], as_index=False)['price'].mean()

# === Pivot to wide format ===
reshaped = grouped.pivot(index='datetime', columns='zone', values='price').reset_index()

# === Save to file ===
output_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\DAM_hourly_cleaned.csv"
reshaped.to_csv(output_path, index=False)

print("✅ DAM reshaped and saved:", output_path)


✅ DAM reshaped and saved: C:\Users\amina.talipova\Desktop\ercot\datasets\DAM_hourly_cleaned.csv


In [18]:
import pandas as pd

# === Load Fuel Mix Data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\FM_combined.csv"
df = pd.read_csv(file_path)

# === Convert to datetime ===
df['datetime'] = pd.to_datetime(df['Datetime'], errors='coerce')
df = df.dropna(subset=['datetime'])

# === Filter for Wind and Solar only ===
df = df[df['Fuel'].isin(['Wind', 'Solar'])]

# === Pivot to wide format: separate columns for Wind and Solar ===
pivoted = df.pivot_table(index='datetime', columns='Fuel', values='MW', aggfunc='mean')

# === Resample to hourly ===
hourly = pivoted.resample('H').mean().reset_index()

# === Rename columns ===
hourly = hourly.rename(columns={'Wind': 'wind_mw', 'Solar': 'solar_mw'})

# === Save to file ===
output_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\FM_hourly_wind_solar.csv"
hourly.to_csv(output_path, index=False)

print("✅ Fuel mix (wind & solar) reshaped and saved:", output_path)


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_1824\1329476502.py:18: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



✅ Fuel mix (wind & solar) reshaped and saved: C:\Users\amina.talipova\Desktop\ercot\datasets\FM_hourly_wind_solar.csv


In [19]:
import pandas as pd

# === Load BESS Data ===
file_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_combined.csv"
df = pd.read_csv(file_path)

# === Parse UTC timestamps ===
df['datetime'] = pd.to_datetime(df['time_utc'], utc=True, errors='coerce')
df = df.dropna(subset=['datetime'])

# === Set datetime index and resample to hourly (sum) ===
df = df.set_index('datetime')
hourly = df.resample('H').sum(numeric_only=True)

# === Rename columns ===
hourly = hourly.rename(columns={
    'total_charging': 'charging_mw',
    'total_discharging': 'discharging_mw',
    'net_output': 'net_output_mw'
}).reset_index()

# === Save to file ===
output_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_hourly.csv"
hourly.to_csv(output_path, index=False)

print("✅ BESS hourly file saved:", output_path)


✅ BESS hourly file saved: C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_hourly.csv


C:\Users\amina.talipova\AppData\Local\Temp\ipykernel_1824\3722194794.py:13: FutureWarning:

'H' is deprecated and will be removed in a future version, please use 'h' instead.



In [22]:
import pandas as pd

# === Load cleaned hourly files ===
rtm = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\RTM_hourly_cleaned.csv", parse_dates=['datetime'])
dam = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\DAM_hourly_cleaned.csv", parse_dates=['datetime'])
fuel = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\FM_hourly_wind_solar.csv", parse_dates=['datetime'])
bess = pd.read_csv(r"C:\Users\amina.talipova\Desktop\ercot\datasets\BESS_hourly.csv", parse_dates=['datetime'])

# === Ensure all datetime columns are timezone-naive ===
bess['datetime'] = bess['datetime'].dt.tz_localize(None)

# === Filter everything from 2016-01-01 onward ===
start_date = pd.Timestamp('2016-01-01')
rtm = rtm[rtm['datetime'] >= start_date]
dam = dam[dam['datetime'] >= start_date]
fuel = fuel[fuel['datetime'] >= start_date]

# === Merge datasets ===
df = rtm.merge(dam, on='datetime', how='outer', suffixes=('_rtm', '_dam'))
df = df.merge(fuel, on='datetime', how='outer')
df = df.merge(bess, on='datetime', how='outer')

# === Sort by datetime ===
df = df.sort_values('datetime')

# === Fill BESS as 0 before 2024 ===
bess_start = pd.Timestamp('2024-01-01')
for col in ['charging_mw', 'discharging_mw', 'net_output_mw']:
    if col in df.columns:
        df[col] = df[col].fillna(0)
        df.loc[df['datetime'] < bess_start, col] = 0

# === Save final dataset ===
output_path = r"C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv"
df.to_csv(output_path, index=False)

print("✅ Final merged dataset saved to:", output_path)


✅ Final merged dataset saved to: C:\Users\amina.talipova\Desktop\ercot\datasets\ERCOT_all_hourly_2016.csv
